In [22]:
import numpy as np
from copy import deepcopy
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from tictactoe import init_board, show_board, check_winner, available_moves

## Vanilla minimax

reg_nodes = 0

In [36]:
def minimax(board, depth, state, quiet = False):
    global reg_nodes
    reg_nodes += 1
    tab = depth*'\t'
    if not quiet:
        print(tab + show_board(board).replace('\n', f'\n{tab}'))
    if check_winner(board):
        if check_winner(board) == 'X':
            return 1
        else:
            return -1
    elif len(available_moves(board)) == 0:
        return 0
    if state == 'X':
        value = -100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = max(value, minimax(board_copy, depth + 1, 'O', quiet = quiet))
        return value
    elif state == 'O':
        value = 100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = min(value, minimax(board_copy, depth + 1, 'X', quiet = quiet))
        return value

## Minimax with alpha beta pruning

In [37]:
alpha_beta_nodes = 0

In [38]:
def alpha_beta_minimax(board, depth, state, alpha, beta, quiet = False):
    '''
    On first call set alpha = -100 and beta = 100
    '''
    global alpha_beta_nodes
    alpha_beta_nodes += 1
    tab = depth*'\t'
    if not quiet:
        print(tab + show_board(board).replace('\n', f'\n{tab}'))
    if check_winner(board):
        if check_winner(board) == 'X':
            return 1
        else:
            return -1
    elif len(available_moves(board)) == 0:
        return 0
    if state == 'X':
        value = -100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = max(value, alpha_beta_minimax(board_copy, depth + 1, 'O', alpha, beta, quiet = quiet))
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value
    elif state == 'O':
        value = 100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = min(value, alpha_beta_minimax(board_copy, depth + 1, 'X', alpha, beta, quiet = quiet))
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

## Comparing the number of iterations

In [39]:
board = init_board()
board[1,1] = 'X'

In [43]:
print(show_board(board))

 | | 
-----
 |X| 
-----
 | | 


In [40]:
minimax(board, 0, 'O', quiet = True)

0

In [41]:
alpha_beta_minimax(board, 0, 'O', -100, 100, quiet=True)

0

In [44]:
print(f'Number of nodes evaluated with regular implementation: {reg_nodes}')
print(f'Number of nodes evaluated with alpha beta pruning: {alpha_beta_nodes}')

Number of nodes evaluated with regular implementation: 55505
Number of nodes evaluated with alpha beta pruning: 2316


## Alpha beta pruning algorithm improves computation quite a bit